Experimenting with multimodal LLMs to interpret graphs

In [1]:
import os
import sys
from dotenv import load_dotenv

## Add root directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

load_dotenv()
assert os.environ["LANGCHAIN_API_KEY"], "Please set the LANGCHAIN_API_KEY environment variable"
assert os.environ["OPENAI_API_KEY"], "Please set the OPENAI_API_KEY environment variable"

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
import base64
from pprint import pprint
from prompts import dashboard_prompts, variable_descriptions

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])
DATA_DIR = "../../../data"

Experiment with multi-agent network with prompt engineer

In [2]:
from man_with_prompt_engineer import MANwithPromptEngineer, create_man_with_prompt_engineer

main_system_message = """
    The user you are assisting today is a trader at an Australian financial institution,
    interested in analyzing some trading data and other matters related to finance and trading.
    Use your team to help him interpret and gather insights from charts on his dashboard. 
"""
db_path = DATA_DIR + "/raw/orders.db"
graph = MANwithPromptEngineer(main_system_message, db_path)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Initializing multi-agent network with prompt engineer...


In [3]:
# image_path_names = [
#     "bi_bar_done_volume.jpg", "bi_bar_price_instruction.jpg", "bi_boxplot_done_volume.jpg", 
#     "bi_pie_exchange.jpg", "bi_pie_lifetime.jpg", "bi_sankey_seccode_buysell.jpg",
#     "asic_pie_directed_wholesale.jpg", "asic_bar_order_capacity.jpg"
# ]

# image_path = DATA_DIR + "/processed/" + image_path_names[0]

# with open(image_path, "rb") as image_file:
#         image_data = base64.b64encode(image_file.read()).decode("utf-8")

# message = HumanMessage(
#     content=[
#         {"type": "text", "text": "This is a bar chart from 1 November 2024."},
#         {
#             "type": "image_url",
#             "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
#         },
#     ],
# )

# # response = graph.invoke_messages([message])
# # pprint(response)
# async for event in graph.graph.astream({"messages": [message]}, config=graph.config):
#     for value in event.values():
#         if "sender" in value:
#             sender = value["sender"]
#         else:
#             sender = "tool"
#         print("-----------------")
#         print(sender)
#         pprint(value["messages"][-1].content)

In [4]:
## Try separating multimodal interpreter from rest of graph
main_system_message = """
The user you are assisting today is a trader at an Australian financial institution,
interested in analyzing some trading data and other matters related to finance and trading.
You are your team need to help him generate descriptions to the chart. 
"""
graph, config = create_man_with_prompt_engineer(llm=openai_llm, main_system_message=main_system_message, db_path=db_path)

In [5]:
image_path_names = [
    "bi_bar_done_volume.jpg", "bi_bar_price_instruction.jpg", "bi_boxplot_done_volume.jpg", 
    "bi_pie_exchange.jpg", "bi_pie_lifetime.jpg", "bi_sankey_seccode_buysell.jpg",
    "asic_pie_directed_wholesale.jpg", "asic_bar_order_capacity.jpg"
]

image_path = DATA_DIR + "/processed/" + image_path_names[0]

with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

messages = [
    SystemMessage(content=dashboard_prompts.system_prompt),
    # SystemMessage(content="These are the variables that you might encounter:\n"+variable_descriptions.variable_descriptions),
    SystemMessage(content=dashboard_prompts.bi_dash_prompt)
]

message = HumanMessage(
    content=[
        {"type": "text", "text": "This is a bar chart from 30 October 2024."},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
        {"type": "text", "text": "Description: "}
    ],
)
messages.append(message)

prelim_response = openai_llm.invoke(messages)

In [6]:
pprint(prelim_response.content)

('### Description of the Bar Chart: Distribution of Done Volume\n'
 '\n'
 'This bar chart presents the distribution of trading volumes, with the x-axis '
 'representing the "Done Volume" (in units) and the y-axis indicating the '
 'count of trades that fall within each volume range.\n'
 '\n'
 '#### Key Features to Note:\n'
 '1. **Peak Count**: The leftmost bar shows a significant peak, indicating '
 'that the majority of trades occurred at lower volume levels, around 0 to '
 '2,000 units, with nearly 900 trades recorded in this range.\n'
 '2. **Volume Distribution**: As volume increases, the count of trades '
 'declines sharply, illustrating a typical distribution where fewer trades '
 'occur at higher volumes.\n'
 '3. **Long Tail**: The chart exhibits a long tail to the right, suggesting '
 'that while high-volume trades are less frequent, they do occur, and may '
 'represent significant market activity.\n'
 '4. **Gap in Mid-Range**: There is a notable dip in trade counts around the '

In [7]:
async for event in graph.astream({"messages": [prelim_response]}, config=config):
    for value in event.values():
        if "sender" in value:
            sender = value["sender"]
        else:
            sender = "tool"
        print("-----------------")
        print(sender)
        pprint(value["messages"][-1].content)
# final_response = graph.invoke({"messages": [prelim_response]}, config=config)
# for message in final_response['messages']:
#     print(message.name)
#     pprint(message.content)
#     print("-------")

-----------------
prompt_engineer
('### Prompts for Assistants\n'
 '\n'
 '#### SQL Assistant\n'
 '**Prompt:** \n'
 '"Please query the database for historical trading volume data over the past '
 'year. Compare the current distribution of done volume with historical trends '
 'to identify any significant changes in trader behavior or market conditions. '
 'Provide insights on how the current distribution aligns or diverges from '
 'previous data."\n'
 '\n'
 '#### Tavily Assistant\n'
 '**Prompt:** \n'
 '"Search the web for articles, studies, or market analysis reports that '
 'discuss trading volume distribution trends in financial markets. Focus on '
 'identifying any insights regarding trader behavior, market conditions, or '
 'potential reasons behind the scarcity of trades in certain volume ranges. '
 'Summarize the findings to complement the insights from the bar chart '
 'analysis."')
-----------------
sql_agent
''
-----------------
tool
'orders'
-----------------
sql_agent
''
----

CancelledError: 

In [8]:
## Combining preliminary and final outputs
prelim_response.content + "\n" + value["messages"][-1].content

'### Chart Description: Distribution of Done Volume\n\nThis bar chart visualizes the distribution of trading volumes for a specific day, allowing traders to understand the frequency of different volume ranges. \n\n**Key Features to Note:**\n\n1. **Count on Y-Axis**: Represents the number of trades that fall within each volume range. A higher count indicates more trades at that volume level.\n2. **Volume Ranges on X-Axis**: Divides the total volume into bins or categories, showing how many trades occurred at each level.\n3. **Distribution Shape**: The distribution appears to have a long tail, suggesting that while many trades occur at lower volumes, fewer trades take place at higher volumes.\n\n**Insights:**\n\n- **Volume Concentration**: The peak at lower volumes indicates that most trades are small, which is typical in many markets. This could suggest a high level of retail trading activity.\n- **Tail Analysis**: The gradual decrease towards higher volumes indicates a potential area f

Experiment with multi-agent network

In [ ]:
from llm.chatbot.chatbot import MultiAgentNetwork

main_system_message = """
    The user you are assisting today is a trader at an Australian financial institution,
    interested in analyzing some trading data and other matters related to finance and trading.
    Help him interpret charts on his dashboard. Use the tools available to you to gather more
    insights to make sense of the charts.
"""
db_path = DATA_DIR + "/raw/orders.db"
graph = MultiAgentNetwork(main_system_message, db_path, llm="openai", agents=["sql"], with_memory=True)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Initializing multi-agent chatbot...


In [14]:
image_path_names = [
    "bi_bar_done_volume.jpg", "bi_bar_price_instruction.jpg", "bi_boxplot_done_volume.jpg", 
    "bi_pie_exchange.jpg", "bi_pie_lifetime.jpg", "bi_sankey_seccode_buysell.jpg",
    "asic_pie_directed_wholesale.jpg", "asic_bar_order_capacity.jpg"
]

image_path = DATA_DIR + "/processed/" + image_path_names[3]

with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

message = HumanMessage(
    content=[
        {"type": "text", "text": "Help me interpret this chart from 1 November 2024."},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)

response = graph.graph.stream({"messages": [message]}, graph.config)

In [ ]:
# pprint(response['messages'][-1].content)
for event in response:
    for value in event.values():
        if "sender" in value:
            sender = value["sender"]
        else:
            sender = "tool"
        print("-----------------")
        print(sender, "\n", value["messages"][-1].content)

Experiment on BI & ASIC dashboard charts

In [ ]:
image_path_names = [
    "bi_bar_done_volume.jpg", "bi_bar_price_instruction.jpg", "bi_boxplot_done_volume.jpg", 
    "bi_pie_exchange.jpg", "bi_pie_lifetime.jpg", "bi_sankey_seccode_buysell.jpg",
    "asic_pie_directed_wholesale.jpg", "asic_bar_order_capacity.jpg"
]

for image_path_name in image_path_names:
    image_path = DATA_DIR + "/processed/" + image_path_name

    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

    messages = [
        SystemMessage(content=dashboard_prompts.system_prompt),
        SystemMessage(content="These are the variables that you might encounter:\n"+variable_descriptions.variable_descriptions)
    ]

    messages.append(
        HumanMessage(
            content=[
                {"type": "text", "text": dashboard_prompts.bi_dash_prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
                },
            ],
        )
    )

    response = openai_llm.invoke(messages)
    print("--------------------")
    print(response.content)

Early experiments with image input

In [ ]:
image_path = DATA_DIR + "/anomaly_detection_sample.png"

with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode("utf-8")

In [12]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "Describe the charts in this dashboard."},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = openai_llm.invoke([message])
print(response.content)

The chart displays a scatter plot titled "Anomaly Detection in Financial Data Across Securities." Here's a breakdown of its components:

- **Axes**: 
  - The x-axis represents the "Price" of the securities, ranging from 0 to 500.
  - The y-axis represents the "Quantity," ranging from 0 to 20,000.

- **Data Points**: 
  - The plot features numerous colored dots, which indicate the quantities of different securities at various price levels. The colors suggest that there are multiple categories or types of securities represented.

- **Anomalies**: 
  - Red 'X' markers highlight the anomalies in the data. These points are likely outliers that deviate significantly from the expected pattern of quantity based on price.

- **Legend**: 
  - There is a legend indicating that the red markers represent "Anomalies."

Overall, the chart visually emphasizes the distribution of financial data across different securities while specifically identifying anomalous data points that may warrant further inv

Add more details in the prompt

In [14]:
import prompts.dashboard_prompts as prompts

messages = [
    SystemMessage(content=prompts.system_prompt)
]

##TODO: Integrate anomalies and reasons
anomalies_and_reasons = {}
anomaly_prompt = prompts.anomaly_dash_prompt.format(anomalies_and_reasons=anomalies_and_reasons)

messages.append(
    HumanMessage(
        content=[
            {"type": "text", "text": anomaly_prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
        ],
    )
)

response = openai_llm.invoke(messages)
print(response.content)

### Chart Description: Anomaly Detection in Financial Data Across Securities

This scatter plot visualizes the results of an anomaly detection model applied to the institution's trading order data. Each point represents a unique trading order, plotted by its price on the x-axis and quantity on the y-axis. The red crosses highlight the flagged anomalies, indicating trades that deviate significantly from typical trading patterns.

Compliance officers can utilize this chart to quickly identify suspicious activities that may warrant further investigation. The distribution of orders across various price ranges, with clusters of flagged anomalies, provides insight into potentially irregular trading behavior. By focusing on these highlighted points, compliance teams can prioritize their review of specific trades that may pose risks to market integrity or compliance with regulatory standards.
